<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Fixa%C3%A7%C3%A3o_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7 and Java 8. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Run a local spark session to test your installation:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

# Reading a CSV from google drive

Utilizando o Google Colab, é possível importar os datasets diretamente do Google Drive, sem ter que realizar o upload manual dos mesmos para a instância colab manualmente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
spark.read\
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("drive/My\ Drive/My\ Professional\ Carrer/Spark\ course/virtual_classroom/colab_classes/data/vgsales.csv") \
  .show()

# Introdução
Para as questões quem envolverem manipulação de Dataframes, utilize o dataset winemag-data_first150k.csv:

https://www.kaggle.com/zynicide/wine-reviews.

Faremos algumas análises utilizando esse dataset.

In [ ]:
csv_path = "winemag-data_first150k.csv"

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("country", StringType(), True),
    StructField("description", StringType(), True),
    StructField("designation", StringType(), True),
    StructField("points", IntegerType(), True),
    StructField("price", DoubleType(), True),
    StructField("province", StringType(), True),
    StructField("region_1", StringType(), True),
    StructField("region_2", StringType(), True),
    StructField("variety", StringType(), True),
    StructField("winery", StringType(), True)
])

df = spark.read.csv(path=csv_path, header="true", schema=schema)
df.show()

+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
| id|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley

# Questão 1
Escreva um conjunto de transformações em Pyspark API e SparkSQL seguido de uma ação `.show()` para exibir a **média das notas (points) obtidos por cada país (country), ordenado decrescentemente pelas notas**:


In [ ]:
# SparkSQL
df.createOrReplaceTempView("wines")

spark.sql("""
SELECT 
  country,
  AVG(points) AS average_points
FROM wines
GROUP BY country
ORDER BY AVG(points) DESC
""").show()

+------------+-----------------+
|     country|   average_points|
+------------+-----------------+
|     England|92.88888888888889|
|     Austria|89.27814136125654|
|      France|88.92595050725325|
|     Germany|88.63183673469388|
|       Italy|88.41341853035144|
|      Canada|88.23979591836735|
|    Slovenia|88.23404255319149|
|     Morocco|88.16666666666667|
|      Turkey|88.09615384615384|
|    Portugal|88.05713211802293|
|   US-France|             88.0|
|     Albania|             88.0|
|   Australia|87.89396081599676|
|          US|87.81835524206477|
|      Serbia|87.71428571428571|
|       India|           87.625|
| New Zealand|87.55562255049743|
|     Hungary|87.32900432900433|
| Switzerland|            87.25|
|South Africa|87.22542072630647|
+------------+-----------------+
only showing top 20 rows



In [ ]:
# Pyspark
import pyspark.sql.functions as f

df.select("country", "points") \
  .groupBy("country") \
  .agg(
      f.avg("points").alias("average_points")
  ) \
  .orderBy(f.col("average_points").desc()) \
  .show()

+------------+-----------------+
|     country|   average_points|
+------------+-----------------+
|     England|92.88888888888889|
|     Austria|89.27814136125654|
|      France|88.92595050725325|
|     Germany|88.63183673469388|
|       Italy|88.41341853035144|
|      Canada|88.23979591836735|
|    Slovenia|88.23404255319149|
|     Morocco|88.16666666666667|
|      Turkey|88.09615384615384|
|    Portugal|88.05713211802293|
|   US-France|             88.0|
|     Albania|             88.0|
|   Australia|87.89396081599676|
|          US|87.81835524206477|
|      Serbia|87.71428571428571|
|       India|           87.625|
| New Zealand|87.55562255049743|
|     Hungary|87.32900432900433|
| Switzerland|            87.25|
|South Africa|87.22542072630647|
+------------+-----------------+
only showing top 20 rows



# Questão 2
Escreva um conjunto de transformações em Pyspark API e SparkSQL seguido de uma ação `.show()` para exibir as **top 20 vinícola (campo winery) mais caras, juntamente com o país e a província**:

In [ ]:
# SparkSQL
df.createOrReplaceTempView("wines")

spark.sql("""
SELECT 
  country,
  province,
  winery,
  AVG(price) AS average_price
FROM wines
GROUP BY country, province, winery
ORDER BY average_price DESC
""").show()

+-------+--------------+--------------------+------------------+
|country|      province|              winery|     average_price|
+-------+--------------+--------------------+------------------+
|     US|    California|               Blair|            1029.0|
| France|      Bordeaux|      Château Latour|             794.4|
|  Italy|       Tuscany|             Masseto|             587.5|
| France|      Bordeaux|  Château Haut-Brion| 569.0909090909091|
| France|      Bordeaux|Château La Missio...| 568.1666666666666|
| France|     Champagne|                Krug|            513.25|
|     US|    California|     Screaming Eagle|             500.0|
| France|      Bordeaux|      Château Ausone|             495.0|
| France|      Burgundy|Domaine Perrot-Minot|            421.25|
| France|      Bordeaux|Château Lafite Ro...|             406.0|
| France|      Bordeaux|     Château Margaux|377.53846153846155|
|  Spain|Northern Spain|        Vega Sicilia|             367.5|
| France|      Bordeaux|C

In [ ]:
import pyspark.sql.functions as f

# Pyspark
df.select("country", "province", "winery", "price") \
  .groupBy("country", "province", "winery") \
  .agg(
      f.avg("price").alias("average_price")
  ) \
  .orderBy(f.col("average_price").desc()) \
  .show()

+-------+--------------+--------------------+------------------+
|country|      province|              winery|     average_price|
+-------+--------------+--------------------+------------------+
|     US|    California|               Blair|            1029.0|
| France|      Bordeaux|      Château Latour|             794.4|
|  Italy|       Tuscany|             Masseto|             587.5|
| France|      Bordeaux|  Château Haut-Brion| 569.0909090909091|
| France|      Bordeaux|Château La Missio...| 568.1666666666666|
| France|     Champagne|                Krug|            513.25|
|     US|    California|     Screaming Eagle|             500.0|
| France|      Bordeaux|      Château Ausone|             495.0|
| France|      Burgundy|Domaine Perrot-Minot|            421.25|
| France|      Bordeaux|Château Lafite Ro...|             406.0|
| France|      Bordeaux|     Château Margaux|377.53846153846155|
|  Spain|Northern Spain|        Vega Sicilia|             367.5|
| France|      Bordeaux|C

# Questão 3
Escreva um conjunto de transformações em Pyspark API e SparkSQL seguido de uma ação `.show()` para exibir as **a média de preços para cada faixa de pontuação de 10 em 10**:

ex: 
```
| faixa | media_preço |
-----------------------
|10 ~ 20|      19     |
|20 ~ 30|      23     |
|...    |      ..     |
|90 ~100|      95     |
```

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

def get_range(number):
  return str(int(number/10)) + '0' + ' ~ ' + str(int(number/10)+1) + '0'

get_range_udf = udf(get_range, StringType())
spark.udf.register("get_range_udf", get_range, StringType())

# SparkSQL
df.createOrReplaceTempView("wines")

spark.sql("""
SELECT 
  get_range_udf(points) as range,
  avg(price)
FROM wines
WHERE points > 0
GROUP BY range
ORDER BY range
""").show()

+---------+------------------+
|    range|        avg(price)|
+---------+------------------+
|100 ~ 110| 401.5833333333333|
|  80 ~ 90|23.770047381361923|
| 90 ~ 100| 53.83318009288728|
+---------+------------------+



In [ ]:
# Pyspark
df.select(get_range_udf("points").alias("range"), "price") \
  .where("points > 0") \
  .groupBy("range") \
  .agg(
      f.avg("price").alias("average_price")
  ) \
  .orderBy(f.col("range").desc()) \
  .show()

+---------+------------------+
|    range|     average_price|
+---------+------------------+
| 90 ~ 100| 53.83318009288728|
|  80 ~ 90|23.770047381361923|
|100 ~ 110| 401.5833333333333|
+---------+------------------+



# Questão 4

Utilize os RDDs para realizar a contagem de cada letra dado o texto abaixo, ou seja, ao final, teremos a seguinte estrutura:
```
('a', 35)
('b', 32)
('c' 5)
...
```

In [ ]:
text = "At a high level, every Spark application consists of a driver program that runs the user’s main function and executes various parallel operations on a cluster. The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing Scala collection in the driver program, and transforming it. Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. Finally, RDDs automatically recover from node failures. A second abstraction in Spark is shared variables that can be used in parallel operations. By default, when Spark runs a function in parallel as a set of tasks on different nodes, it ships a copy of each variable used in the function to each task. Sometimes, a variable needs to be shared across tasks, or between tasks and the driver program. Spark supports two types of shared variables: broadcast variables, which can be used to cache a value in memory on all nodes, and accumulators, which are variables that are only added to, such as counters and sums. This guide shows each of these features in each of Spark’s supported languages. It is easiest to follow along with if you launch Spark’s interactive shell – either bin/spark-shell for the Scala shell or bin/pyspark for the Python one."
list_text = text.split(" ")

In [ ]:
list("igor")

['i', 'g', 'o', 'r']

In [ ]:
sc = spark.sparkContext

rdd = sc.parallelize(list_text)

rdd.flatMap(lambda word: list(word)) \
  .map(lambda letter: (letter.lower(), 1)) \
  .reduceByKey(lambda a, b: a + b) \
  .sortBy(lambda pair: pair[0]) \
  .take(20)
# ...

[('(', 2),
 (')', 2),
 (',', 14),
 ('-', 2),
 ('.', 11),
 ('/', 2),
 (':', 1),
 ('a', 132),
 ('b', 20),
 ('c', 44),
 ('d', 47),
 ('e', 128),
 ('f', 28),
 ('g', 12),
 ('h', 47),
 ('i', 82),
 ('k', 15),
 ('l', 63),
 ('m', 20),
 ('n', 73)]

# Questão 5

Imagine que você foi contratado como engenheiro de dados para construir uma aplicação que seja capaz de processar dados em near real-time relativos a transações bancárias em um banco X.

Basicamente, as trasações ocorrem enquanto o usuário utiliza o app do banco X. A cada transação, esses dados são enviados para um servidor em nuvém que irá recebê-los e encaminhar para o sistema de streaming classificá-los.

Esse streaming servirá de base para que uma equipe anti-fraude, do banco X, seja informada sempre que uma transação suspeita ocorrer e tomará ações o mais rápido o possível. Assim sendo, o **streaming não será ativo, ou seja, não tomará ações. Ele apenas informará em real-time sobre uma possível fraude em algum lugar**.

**BONUS**: Em paralelo a isso, para que seja possível gerar um streaming preciso, um modelo de machine learning é alimentado diariamente com os históricos dos últimos 5 anos das transações bancárias, consideradas fraudes ou não. Esse modelo é treinado diariamente e o modelo gerado é disponibilizado para o streaming, sempre que possível.

Como você desenharia a arquitetura dessa solução? Quais os melhores componentes? Como seria o seu pipeline de dados?

Descreva utilizando um desenho. Pode mandar uma foto do desenho (a caneta) ou usar uma ferramenta web para desenhar a arquitura (draw.io). Favor enviar diretamente para o professor no Slack.

# Questão 6

Descreva o que são **accumulators** e **broadcast variables** e explique quando e porque utilizá-las.


# Questão 7

Utilizando o dado proveniente do Kafka, já visto em sala de aula, implemente um contador de transações em streaming para cada **conta destino** (campo target). Crie duas versões do streaming:

- utilizando Structured Streaming
- utilizando Spark Streaming

Ex.:
```
# target, coutn
('aZ', 123)
('as', 43)
```

In [ ]:
# Structured Streaming
qnt_sources_df = identified_df \
  .groupBy("target") \
  .count()

In [ ]:
query = qnt_sources_df \
  .writeStream \
  .outputMode("complete") \
  .format("memory") \
  .queryName("qnt_target") \
  .start()

In [ ]:
# Spark Streaming
def basic_transformation(dks):
    dks \
        .map(lambda x: (json.loads(x[1])["target"], 1)) \
        .reduceByKey(lambda a,b: a+b, numPartitions=2) \
        .pprint()

basic_transformation(dks)

#Starting Spark context
ssc.checkpoint("checkpoint_streaming")
ssc.start()
ssc.awaitTermination()